In [1]:
import pandas as pd
import numpy as np
import os, zipfile
import subprocess

In [2]:
# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
import lightgbm as lgbm
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgbm
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [3]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [4]:
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

In [5]:
from imblearn.under_sampling import RandomUnderSampler

In [6]:
import category_encoders as ce

In [7]:
dir_name = 'D:\\1mfti\ml\Home-Credit-Default-Risk'
os.chdir(dir_name)
train = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')

In [8]:
num_cols = list(train.select_dtypes(include=['int64', 'float64']).columns)
num_cols.remove('TARGET')

In [9]:
cat_cols = list(train.select_dtypes(include=['object']).columns)

In [10]:
def preprocess(train1, test1):
    train1 = train1.fillna(-10)
    test1 = test1.fillna(-10)
    train1[num_cols] = train1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    test1[num_cols] = test1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    WOE_encoder = ce.WOEEncoder()
    train1[cat_cols] = WOE_encoder.fit_transform(train1[cat_cols], train1['TARGET'])
    test1[cat_cols] = WOE_encoder.transform(train1[cat_cols])

    return train1, test1

In [12]:
train_norm, test_norm = preprocess(train, test)

In [13]:
X_train = train_norm.drop('TARGET', axis=1)
y_train = train_norm['TARGET']
X_test = test_norm

In [14]:
ada = RandomUnderSampler(random_state=42)
X_res, y_res = ada.fit_resample(X_train, y_train)

In [ ]:
# X_res, y_res = X_res[:20000], y_res[:20000]

In [15]:
X_res, X_valid, y_res, y_valid = train_test_split(X_res, y_res,
                                                    stratify=y_res, 
                                                    test_size=0.2)

In [19]:
lgbm_clf = lgbm.LGBMClassifier(objective='binary', 
         boosting_type = 'goss', 
         n_estimators = 50, class_weight ='balanced', learning_rate =0.01, random_state=42, eval_set=[X_valid, y_valid])

In [ ]:
lgbm_clf.fit(X_res, y_res)

In [ ]:
# Точность на kaggle:0.57896 learning_rate=0.01, n_estimators=50, objective='binary',
               random_state=42)

In [88]:


clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    n
    
    #loss_function='CrossEntropy'
)
#Method 1
grid = dict()
grid['n_estimators'] = [100, 200, 300]
grid['max_depth'] = [3,4,5]
Method 2
grid = {'max_depth': [3,4,5],'n_estimators':[100, 200, 300]}
#Instantiate GridSearchCV
gscv = GridSearchCV (estimator = cbc, param_grid = grid, scoring ='accuracy', cv = 5)


clf.fit(X_res, y_res, 
        
        eval_set=(X_valid, y_valid), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 5, 'learning_rate': 0.1}


In [ ]:

cbc = CatBoostClassifier()

# Creating the hyperparameter grid
param_dist = { "learning_rate": np.linspace(0,0.2,5),
               "max_depth": sp_randint(3, 10),
            "learning_rate": [0.1, 0.01, 0.2, 0.3]},
    
               
#Instantiate RandomSearchCV object
rscv = RandomizedSearchCV(cbc , param_dist, scoring='roc_auc', cv =5)

#Fit the model
rscv.fit(X_res,y_res)

# Print the tuned parameters and score
print(rscv.best_params_)
print(rscv.best_score_)

In [110]:
print(rscv.best_params_)
print(rscv.best_score_)

{'learning_rate': 0.01, 'max_depth': 8}
0.6725260167091087


In [ ]:
#Точность на kaggle: 0.57147  

In [16]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)))
    level0.append(('knn', KNeighborsClassifier()))
    level0.append(('bbc', BalancedBaggingClassifier()))
    level0.append(('svm', SVC()))
    level0.append(('bayes', GaussianNB()))
    # level0.append(('lama', automl))
    level0.append(('lgbm', lgbm_clf))
    # define meta learner model
    level1 = LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [17]:
def get_kaggle_results():
    p = subprocess.Popen("kaggle competitions submissions  -c home-credit-default-risk", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    print(output.decode())

In [20]:
stacking = get_stacking()

In [ ]:
stacking.fit(X_res, y_res)

In [29]:
predict_2 = stacking.predict_proba(X_test)

In [33]:
predict_2 = 1- predict_2

In [36]:
write_submission_2 = pd.read_csv('data/sample_submission.csv')
write_submission_2['TARGET'] = predict_2
write_submission_2.to_csv('predict_stacking.csv', index= False)

In [111]:
predict_cat_grid = rscv.predict(X_test)

In [112]:
write_submission_2 = pd.read_csv('data/sample_submission.csv')
write_submission_2['TARGET'] = predict_cat_grid
write_submission_2.to_csv('predict_cat_grid.csv', index= False)

In [37]:
!kaggle competitions submit -c home-credit-default-risk -f predict_stacking.csv -m "lgbm"

Successfully submitted to Home Credit Default Risk


  0%|          | 0.00/1.27M [00:00<?, ?B/s]
  1%|          | 8.00k/1.27M [00:00<00:45, 28.9kB/s]
  7%|7         | 96.0k/1.27M [00:00<00:04, 248kB/s] 
 15%|#4        | 192k/1.27M [00:00<00:02, 381kB/s] 
 30%|##9       | 384k/1.27M [00:00<00:01, 722kB/s]
 58%|#####8    | 760k/1.27M [00:00<00:00, 1.49MB/s]
 73%|#######3  | 952k/1.27M [00:01<00:00, 1.31MB/s]
 85%|########5 | 1.09M/1.27M [00:01<00:00, 1.31MB/s]
 98%|#########7| 1.24M/1.27M [00:01<00:00, 1.31MB/s]
100%|##########| 1.27M/1.27M [00:03<00:00, 374kB/s] 


In [ ]:
get_kaggle_results()

In [ ]:
Точность стэкинга 0.62

Уберем лучшую модель

In [57]:
def predict(predict, name):
    write_submission_2 = pd.read_csv('data/sample_submission.csv')
    write_submission_2['TARGET'] = predict
    filename = f'prediction_model.csv'
    write_submission_2.to_csv(filename, index= False)
    

In [ ]:
stack_model_names = ['lr', 'knn', 'bbc', 'bayes', 'lgbm']
for model in stack_model_names:
    prediction = stacking.named_estimators_[model].predict_proba(X_test)
    prediction = 1- prediction
    predict(prediction, model)
    !kaggle competitions submit -c home-credit-default-risk -f prediction_model.csv -m f"{model}"
    

## Результаты моделей по отдельности

    lgbm        complete  0.60471      0.60297       
    bbc         complete  0.55528      0.55769       
    knn         complete  0.56989      0.56248       
    lr          complete  0.63152      0.62960  

In [62]:
# Стэкинг без лр
def get_stacking2():
    # define the base models
    level0 = list()
#     level0.append(('lr', LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)))
    level0.append(('knn', KNeighborsClassifier()))
    level0.append(('bbc', BalancedBaggingClassifier()))
    level0.append(('svm', SVC()))
    level0.append(('bayes', GaussianNB()))
    # level0.append(('lama', automl))
    level0.append(('lgbm', lgbm_clf))
    # define meta learner model
    level1 = LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [63]:
stacking = get_stacking2()

In [ ]:
stacking.fit(X_train, y_train)

In [ ]:
predict_2 = stacking.predict_proba(X_test)
predict_2 = 1- predict_2

In [ ]:
write_submission_2 = pd.read_csv('data/sample_submission.csv')
write_submission_2['TARGET'] = predict_2
write_submission_2.to_csv('predict_stacking2.csv', index= False)

In [ ]:
!kaggle competitions submit -c home-credit-default-risk -f predict_stacking2.csv -m "lgbm"